In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

class LDA():
    _feat = None
    _targ = None
    _N = None
    _p_one = None
    _p_zero = None
    _u_one = None
    _u_zero = None
    _E_one = None
    _E_zero = None
    _E = None
    def __init__(self,features,targets):
        self._feat = features
        self._targ = targets
        self._N = targets.size #Expecting 1d Array of labels
        self._n_one = targets.sum()
        self._n_zero = (self._N-self._targ.sum())
        self._p_one = self._n_one/self._N
        self._p_zero = self._n_zero/self._N
        self._u_zero = 0
        self._u_one = 0
        for x in range(self._N): ##INDICATOR
            if self._targ[x] == 1:
                self._u_one += self._feat[x]
            else:
                self._u_zero += self._feat[x]
        self._u_zero = self._u_zero/self._n_zero #Expecting 1/0 labels passed on
        self._u_one = self._u_one/self._n_one
        self._E_one =0
        self._E_zero =0
        for x in range(self._N):
            if self._targ[x] == 1:
                self._E_one += np.outer((self._feat[x] - self._u_one),(self._feat[x] - self._u_one))/(self._N -2)
            else:
                self._E_zero += np.outer((self._feat[x] - self._u_zero),(self._feat[x] - self._u_zero))/(self._N -2)
        self._E = self._E_one + self._E_zero

    def predict(self,features):
        predResult = []
        a = np.log(self._p_one/self._p_zero)
        b = (np.dot(np.dot(self._u_one.T,np.linalg.inv(np.array(self._E,dtype=np.float32))),self._u_one)/2) # Bit weird to explain why I didnt power by -1
        print(b)
        c = (np.dot(np.dot(self._u_zero.T,np.linalg.inv(np.array(self._E,dtype=np.float32))),self._u_zero)/2)
        print(c)
        for x in range(features.shape[0]):
            d = np.dot(np.dot(features[x].T,np.linalg.inv(np.array(self._E,dtype=np.float32))),(self._u_one-self._u_zero))
#            d = ((features[x].T*np.power(self._E,-1)*(self._u_one-self._u_zero)))
            res = a-b+c+d
            if res > 0:
                predResult.append(1)
            else:
                predResult.append(0)
            
        return np.array(predResult)
    
    def Accu_eval(self,pL,tL):
        n=tL.size
        print(tL,pL)
        return np.mean(pL == tL)
    


In [2]:
main()

1218.659820869128
1246.094612848161
[1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1
 1 1] [1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1
 1 1]
0.8974358974358975


In [5]:
class LogisticRegression():
    #Attributes
    _features = None
    _targets = None
    _weights = None
    featt = None
    tarr = None
    
    def __init__(self, feat,target):
        #TARGET IS EXPECTED TO BE changed to 1d array of 1 and zeros
        feat = np.insert(feat,0,1,axis=1) ##Adding x^0
        self._features = feat
        self._weights = np.zeros(self._features[0].size)
        self._targets = target
        self.tarr = self._targets.flatten()
        self.featt = self._features

    def fit(self, tData, corVec, lRate, itera): 
        for x in range(itera):
            N = tData.shape[0]
            grad = np.dot((corVec-self.sig(np.dot(tData,self._weights))),tData) 
            grad *= lRate
            grad /= N
            self._weights = np.add(self._weights,grad)


    def predict(self,tSet):
        return self._classify(self.sig(np.array(np.dot(tSet, self._weights),dtype=np.float32)))
    
    def cost(self,trueV,predV):
        obs = trueV.shape[0]
        
        return ((-trueV*np.log(predV))-((1-trueV)*np.log(1-predV))).sum()/obs
    def sig(self,r):
        return 1.0/(1.0+np.exp(-(np.array(r,dtype=np.float32))))
    
    def printW(self):
        return self._weights
    def _classify(self,pred):
        l = []
        for x in pred:
            if x >= 0.5:
                l.append(1)
            else:
                l.append(0)
        return np.array(l)
        
    def Accu_eval(self,pL,tL):
        n=tL.size
        return np.mean(pL == tL)

In [13]:
df1 = pd.read_csv('parkinsons.csv')
df2 = pd.read_csv('sonar.csv')

In [14]:
df1_y = df1['status']
df1_x = df1.drop(['name', 'status'], axis=1)


In [18]:
from sklearn.model_selection import train_test_split



In [19]:
X_train, X_test, y_train, y_test = train_test_split(df1_x, df1_y, test_size=0.2)

In [26]:
import time
start_time = time.time()
lr_clf = LogisticRegression(np.array(X_train), np.array(y_train))
lr_clf.fit(lr_clf.featt,lr_clf.tarr,0.3,1000)
pred = lr_clf.predict(np.insert(np.array(X_test),0,1,axis=1))
print("--- %s seconds for Logistic Regression prediction ---" % (time.time() - start_time))
print(lr_clf.Accu_eval(pred, np.array(y_test)))

--- 0.03148674964904785 seconds for Logistic Regression prediction ---
0.7692307692307693


In [28]:
start_time1 = time.time()
lda_clf = LDA(np.array(X_train), np.array(y_train))
pred = lda_clf.predict(np.array(X_test))
print("--- %s seconds for LDA prediction ---" % (time.time() - start_time1))
print(lda_clf.Accu_eval(np.array(pred), np.array(y_test)))

1153.2558031932604
1176.508625973147
--- 0.005982160568237305 seconds for LDA prediction ---
[1 1 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1
 0 0] [1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1
 1 1]
0.8461538461538461


In [30]:
df2.head()
df2_y = df2['R']
df2_x = df2.drop(['R'], axis=1)


In [32]:
binary_encoding = {'R' : 1, 'M' : 0}
df2_y.replace(binary_encoding, inplace=True)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(df2_x, df2_y, test_size=0.2)

start_time = time.time()
lr_clf = LogisticRegression(np.array(X_train), np.array(y_train))
lr_clf.fit(lr_clf.featt,lr_clf.tarr,0.3,1000)
pred = lr_clf.predict(np.insert(np.array(X_test),0,1,axis=1))
print("--- %s seconds for Logistic Regression prediction ---" % (time.time() - start_time))
print(lr_clf.Accu_eval(pred, np.array(y_test)))


start_time1 = time.time()
lda_clf = LDA(np.array(X_train), np.array(y_train))
pred = lda_clf.predict(np.array(X_test))
print("--- %s seconds for LDA prediction ---" % (time.time() - start_time1))
print(lda_clf.Accu_eval(np.array(pred), np.array(y_test)))

--- 0.02096080780029297 seconds for Logistic Regression prediction ---
0.8095238095238095
57.145884065421
64.58266473990804
--- 0.01798272132873535 seconds for LDA prediction ---
[1 0 1 0 1 1 0 0 1 0 0 0 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0
 0 0 1 0 1] [1 1 1 0 1 0 0 0 1 0 0 0 1 0 1 1 1 1 0 1 0 0 1 0 1 1 0 1 1 0 0 0 0 1 0 0 0
 0 1 1 0 0]
0.7857142857142857


In [36]:
learning_rates = [0.1, 0.2, 0.3, 0.01, 0.02, 0.05, 0.001, 0.0001]
def test_logreg_lrs(X_train, X_test, y_train, y_test, lrs):
    for lr in lrs:
        start_time = time.time()
        lr_clf = LogisticRegression(np.array(X_train), np.array(y_train))
        print("Testing Learning rate: ", lr)
        lr_clf.fit(lr_clf.featt,lr_clf.tarr,lr, 1000)
        pred = lr_clf.predict(np.insert(np.array(X_test),0,1,axis=1))
        print("--- %s seconds for Logistic Regression prediction ---" % (time.time() - start_time))
        print(lr_clf.Accu_eval(pred, np.array(y_test)))

In [37]:
test_logreg_lrs(X_train, X_test, y_train, y_test, learning_rates)

Testing Learning rate:  0.1
--- 0.019970178604125977 seconds for Logistic Regression prediction ---
0.8095238095238095
Testing Learning rate:  0.2
--- 0.016956090927124023 seconds for Logistic Regression prediction ---
0.8095238095238095
Testing Learning rate:  0.3
--- 0.018957853317260742 seconds for Logistic Regression prediction ---
0.8095238095238095
Testing Learning rate:  0.01
--- 0.014959573745727539 seconds for Logistic Regression prediction ---
0.6666666666666666
Testing Learning rate:  0.02
--- 0.01395726203918457 seconds for Logistic Regression prediction ---
0.7857142857142857
Testing Learning rate:  0.05
--- 0.015074491500854492 seconds for Logistic Regression prediction ---
0.7857142857142857
Testing Learning rate:  0.001
--- 0.0 seconds for Logistic Regression prediction ---
0.5238095238095238
Testing Learning rate:  0.0001
--- 0.0156402587890625 seconds for Logistic Regression prediction ---
0.5238095238095238


In [38]:
X_train, X_test, y_train, y_test = train_test_split(df1_x, df1_y, test_size=0.2)
test_logreg_lrs(X_train, X_test, y_train, y_test, learning_rates)


Testing Learning rate:  0.1
--- 0.022917747497558594 seconds for Logistic Regression prediction ---
0.8205128205128205
Testing Learning rate:  0.2
--- 0.02695488929748535 seconds for Logistic Regression prediction ---
0.2564102564102564
Testing Learning rate:  0.3
--- 0.019917964935302734 seconds for Logistic Regression prediction ---
0.8205128205128205
Testing Learning rate:  0.01
--- 0.014914989471435547 seconds for Logistic Regression prediction ---
0.1794871794871795
Testing Learning rate:  0.02
--- 0.019982099533081055 seconds for Logistic Regression prediction ---
0.8205128205128205
Testing Learning rate:  0.05
--- 0.02023029327392578 seconds for Logistic Regression prediction ---
0.1794871794871795
Testing Learning rate:  0.001
--- 0.008060932159423828 seconds for Logistic Regression prediction ---
0.8205128205128205
Testing Learning rate:  0.0001
--- 0.015620708465576172 seconds for Logistic Regression prediction ---
0.8205128205128205


In [39]:
parkinsons_df = pd.read_csv('parkinsons.csv')

In [41]:
parkinsons_df.columns


Index(['name', 'MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)',
       'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 'Jitter:DDP',
       'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5',
       'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'status', 'RPDE', 'DFA',
       'spread1', 'spread2', 'D2', 'PPE'],
      dtype='object')

In [44]:
parkinsons_new = parkinsons_df.loc[:, ['MDVP:Jitter(%)','spread1', 'spread2', 'D2', 'PPE', 'RPDE']]

In [45]:
parkinsons_y = parkinsons_df['status']

In [46]:
X_train, X_test, y_train, y_test = train_test_split(parkinsons_new, parkinsons_y, test_size=0.2) 

In [47]:
start_time = time.time()
lr_clf = LogisticRegression(np.array(X_train), np.array(y_train))
lr_clf.fit(lr_clf.featt,lr_clf.tarr,0.3,1000)
pred = lr_clf.predict(np.insert(np.array(X_test),0,1,axis=1))
print("--- %s seconds for Logistic Regression prediction ---" % (time.time() - start_time))
print(lr_clf.Accu_eval(pred, np.array(y_test)))

--- 0.015971899032592773 seconds for Logistic Regression prediction ---
0.8974358974358975


In [48]:
start_time1 = time.time()
lda_clf = LDA(np.array(X_train), np.array(y_train))
pred = lda_clf.predict(np.array(X_test))
print("--- %s seconds for LDA prediction ---" % (time.time() - start_time1))
print(lda_clf.Accu_eval(np.array(pred), np.array(y_test)))

487.7736169411521
503.8918588132939
--- 0.016470670700073242 seconds for LDA prediction ---
[1 1 1 0 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0 1
 1 0] [1 1 1 0 1 0 1 1 0 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 1
 1 1]
0.8974358974358975
